In [1]:
try:
    import sys
    import os
    sys.path.insert(0, os.path.abspath(os.curdir))
except ModuleNotFoundError:
    pass
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.sql.functions as f
import pyspark.sql.types as t
import os
from functools import reduce
from datetime import datetime, time
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag
from pyspark.sql.window import Window
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType
import pandas as pd
import math
import folium
import carregar_mapa_desagrupados as cmd
import carregar_dados_agrupados as cda


<p>1 - Iniciar sessão spark</p>

In [2]:
spark_session = cda.iniciar_sessao_spark()

23/09/30 18:22:48 WARN Utils: Your hostname, DESKTOP-N81ME1R resolves to a loopback address: 127.0.1.1; using 172.24.108.240 instead (on interface eth0)
23/09/30 18:22:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/30 18:22:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
dataframe_desagrupados = cmd.load_dataframe_desagrupados(spark=spark_session)
dataframe_desagrupados.show()

+-----------+--------------------+--------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|CODIGO_AREA|            CONSOCIO|             EMPRESA|DATA_EXTRACAO|HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|           LATITUDE|          LONGITUDE|DATA_EXTRACAO_API|TURNO|
+-----------+--------------------+--------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|          4|PÊSSEGO TRANSPORT...|PÊSSEGO TRANSPORT...|   2023-09-15|   08:50|          3054-10|               2|                     33743|         47399|         2023-09-15T11:50:12Z|         -23.598638|         -46.416157|       2023-09-15|Manhã|


In [4]:
df_prefixo_onibus = cmd.obter_prefixo_onibus(dataframes_desagrupados_completo=dataframe_desagrupados)
df_prefixo_onibus

[66580,
 15173,
 73951,
 78598,
 26486,
 61679,
 36102,
 31944,
 66070,
 11567,
 48280,
 22129,
 31703,
 15057,
 26543,
 77605,
 36103,
 45298,
 55474,
 66114,
 63212,
 31156,
 64442,
 71655,
 32098,
 68463,
 56242,
 52743,
 72935,
 52001,
 10422,
 61749,
 71530,
 48899,
 51418,
 21965,
 63676,
 52910,
 51428,
 64317,
 31762,
 11745,
 51056,
 72485,
 15194,
 45166,
 72926,
 45410,
 25207,
 16530,
 55426,
 45726,
 71429,
 16597,
 85760,
 52051,
 71285,
 71292,
 47745,
 71386,
 81572,
 36077,
 47050,
 71549,
 16673,
 81443,
 48963,
 52904,
 76085,
 77441,
 71548,
 45390,
 26140,
 85641,
 15221,
 68166,
 68047,
 63181,
 63448,
 86153,
 66911,
 73916,
 71255,
 51105,
 68752,
 76050,
 11771,
 47741,
 48203,
 63678,
 45724,
 63586,
 81038,
 55329,
 55093,
 77434,
 11860,
 45558,
 68314,
 51112,
 72928,
 31159,
 32204,
 47683,
 15133,
 11988,
 31111,
 47636,
 63109,
 47229,
 78203,
 52745,
 22808,
 31126,
 68608,
 45134,
 31102,
 80978,
 31832,
 36577,
 45473,
 16499,
 22707,
 45580,
 45474,


In [5]:
prefixo_onibus = '22129'

In [6]:
dataframe_filter = cmd.dataframe_filter_desagrupados(
    dataframes_desagrupados_completo=dataframe_desagrupados,
    data_extracao='2023-09-18',
    prefixo_onibus=prefixo_onibus,
    turno='Noite'
)
dataframe_filter.show()

+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|   18:00|         22129|                       709|          971D-10|     -23.5131826875|        -46.6161215|
|   18:05|         22129|                       709|          971D-10|     -23.5129060625|        -46.6160925|
|   18:10|         22129|                       709|          971D-10|     -23.5131826875|        -46.6161215|
|   18:15|         22129|                     33477|          971D-10|-23.513426000000003|-46.615978999999996|
|   18:20|         22129|                     33477|          971D-10|-23.509598875000002|       -46.61557775|
|   18:40|         22129|                     33477|          971D-10|       -23.49238875|         -46.626027|
|

In [7]:
letreiro_onibus = cmd.obter_letreiro_onibus(dataframe_prefixo_onibus=dataframe_filter)
letreiro_onibus

'971D-10'

In [8]:
rota_linha = cmd.obter_rota_linha(letreiro_onibus=letreiro_onibus)
rota_linha[0]

['971D-10', 'Jd. Damasceno - Shop. Center Norte', '002F6C']

In [9]:
dataframe_filter = dataframe_filter.withColumn("DISTANCIA", cmd.haversine_udf(dataframe_filter["LATITUDE"], dataframe_filter["LONGITUDE"]))
dataframe_filter.show()

+--------+--------------+--------------------------+-----------------+-------------------+-------------------+--------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|           DISTANCIA|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+--------------------+
|   18:00|         22129|                       709|          971D-10|     -23.5131826875|        -46.6161215|                 0.0|
|   18:05|         22129|                       709|          971D-10|     -23.5129060625|        -46.6160925|0.030901094296663396|
|   18:10|         22129|                       709|          971D-10|     -23.5131826875|        -46.6161215|0.030901094296663396|
|   18:15|         22129|                     33477|          971D-10|-23.513426000000003|-46.615978999999996|0.030709748535673966|
|   18:20|         22129|                     33477|          971D-10|-23.50

In [10]:
soma_distancias = dataframe_filter.selectExpr("sum(DISTANCIA)").collect()[0][0]
print(f'Soma das distâncias: {soma_distancias:.2f} km')

Soma das distâncias: 36.03 km


In [11]:
rota_linha

[['971D-10', 'Jd. Damasceno - Shop. Center Norte', '002F6C']]

In [12]:
id_tracado_linha = cmd.obter_id_tracado_linha(spark_session=spark_session, rota_id=rota_linha[0][0])
id_tracado_linha

[72660, 72661]

In [13]:
tracado_linha = cmd.obter_tracado_linha(spark_session=spark_session,shape_id=id_tracado_linha)
tracado_linha

[[-23.447513580322266, -46.69690704345703],
 [-23.447511672973633, -46.69706344604492],
 [-23.447507858276367, -46.697174072265625],
 [-23.447511672973633, -46.69729232788086],
 [-23.447519302368164, -46.697391510009766],
 [-23.447526931762695, -46.69743347167969],
 [-23.44754981994629, -46.69749069213867],
 [-23.447574615478516, -46.69753646850586],
 [-23.447620391845703, -46.69758224487305],
 [-23.447654724121094, -46.69761276245117],
 [-23.447681427001953, -46.697628021240234],
 [-23.447704315185547, -46.69763946533203],
 [-23.4477596282959, -46.69764709472656],
 [-23.44792366027832, -46.697654724121094],
 [-23.448020935058594, -46.697654724121094],
 [-23.4481201171875, -46.697669982910156],
 [-23.448192596435547, -46.69769287109375],
 [-23.448219299316406, -46.69770812988281],
 [-23.448251724243164, -46.69772720336914],
 [-23.44835090637207, -46.69780731201172],
 [-23.448436737060547, -46.6978874206543],
 [-23.4484920501709, -46.697940826416016],
 [-23.44851303100586, -46.697959899

In [14]:
def obter_posicao_onibus_teste(df_posicao: DataFrame):
    posicao_onibus = df_posicao.rdd.flatMap(
        lambda linha: [[(linha.HORA_API), [linha.LATITUDE, linha.LONGITUDE]]]
    ).collect()

    return posicao_onibus

In [15]:
posicao_onibus = cmd.obter_posicao_onibus(df_posicao=dataframe_filter)
posicao_onibus[0]

['18:00', [-23.5131826875, -46.6161215]]

In [16]:
rota_linha

[['971D-10', 'Jd. Damasceno - Shop. Center Norte', '002F6C']]

In [17]:
mapa = cmd.gerar_mapa(posicao_onibus=posicao_onibus, tracado_linha=tracado_linha,cor_linha=rota_linha[0][2], prefixo_onibus=prefixo_onibus)
mapa